In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import gensim
import spacy
import texthero as hero
import random
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
data1_src = "data1/News _dataset/"


In [5]:
data1_fake ="Fake.csv"
df_fake = pd.read_csv(data1_src + data1_fake)
df_fake.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
df_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [7]:
df_fake.describe()

,title,text,subject,date
count,23481,23481,23481,23481
unique,17903,17455,6,1681
top,MEDIA IGNORES Time That Bill Clinton FIRED His...,,News,"May 10, 2017"
freq,6,626,9050,46


In [8]:
df_fake.groupby(['subject']).count()

,title,text,date
subject,,,
Government News,1570,1570,1570
Middle-east,778,778,778
News,9050,9050,9050
US_News,783,783,783
left-news,4459,4459,4459
politics,6841,6841,6841


The above caegories, except for 'Middle-east' news, may have overlapping / similar news stories. Hencetwo categories will be selected to match a similar category in true news data.

In [9]:

fnews = (df_fake['subject']=='politics' )| (df_fake['subject'] == 'US_News')
fdf = df_fake[fnews]


Only poltical news stories are going to be used to keep the data uniform.Tweets have shorter legth than stories.

In [10]:
ftweets = df_fake['title'].str.contains("(TWEETS)", case = False) == False
f_df = fdf[ftweets]

/usr/local/lib/python3.9/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
<ipython-input-10-7b14306f9e1c>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  f_df = fdf[ftweets]


There do not seem to be a number of tweets in the dataset. 

In [11]:
f_df['target'] = 0

<ipython-input-11-bae77650778f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_df['target'] = 0


In [12]:
f_df = f_df[['text', 'target']]
## 
f_df = f_df.reset_index(drop = True)

In [13]:
f_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7585 entries, 0 to 7584
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7585 non-null   object
 1   target  7585 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 118.6+ KB


In [14]:
f_df.head()

,text,target
0,"According to The Hill, Democrat Senator Bob Ca...",0
1,If we protect every other government building ...,0
2,Ami Horowitz is fantastic! Check out this man ...,0
3,Just one more reminder of why President Trump ...,0
4,Thank goodnesss Jeff Sessions is moving on fin...,0


That marks the basic extrtaction of "fake news" dataset. It has to be processed further.

Now, it is time to take a look at the True dataset.

In [15]:
data1_true = "True.csv"
df_true = pd.read_csv(data1_src + data1_true)
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [16]:
df_true.dropna(inplace = True)

In [17]:
df_true.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 836.6+ KB


In [18]:
df_true.describe()

,title,text,subject,date
count,21417,21417,21417,21417
unique,20826,21192,2,716
top,Factbox: Trump fills top jobs for his administ...,(Reuters) - Highlights for U.S. President Dona...,politicsNews,"December 20, 2017"
freq,14,8,11272,182


In [19]:
df_true.groupby(['subject']).count()

,title,text,date
subject,,,
politicsNews,11272,11272,11272
worldnews,10145,10145,10145


Unlike the fake news dataset, true news is gathered only from Reuters and has only two categories. It would be a good idea to consider only US political news in order to look as homogenous as possible with the fake news data, in terms of the topics.

In [20]:
tnews = df_true['subject'] == 'politicsNews'
truenews_df = df_true[tnews]
truenews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11272 entries, 0 to 11271
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    11272 non-null  object
 1   text     11272 non-null  object
 2   subject  11272 non-null  object
 3   date     11272 non-null  object
dtypes: object(4)
memory usage: 440.3+ KB


In [21]:
truenews_df.describe()

,title,text,subject,date
count,11272,11272,11272,11272
unique,11150,11214,1,716
top,Factbox: Trump fills top jobs for his administ...,(Reuters) - Highlights for U.S. President Dona...,politicsNews,"November 9, 2016"
freq,14,8,11272,115


When Reuters posts are checked it will have a signature mark of "(Reuters)" at the beginning of the news story.

In [22]:
truenews_df = truenews_df.reset_index(drop= True)
truenews_df['target'] = 1


In [23]:
tdf = truenews_df['text'].str.contains("(Reuters)",case = False)==True
t_df = truenews_df[tdf]
t_df.dropna(inplace = True)

/usr/local/lib/python3.9/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
<ipython-input-23-ea367179c4b5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_df.dropna(inplace = True)


In [24]:
t_df = t_df[['text', 'target']]

In [25]:
t_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11239 entries, 0 to 11271
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    11239 non-null  object
 1   target  11239 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 263.4+ KB


In [26]:
t_df = t_df.reset_index(drop= True)

In [27]:
t_df['text'] = hero.clean(t_df['text'])

/usr/local/lib/python3.9/site-packages/texthero/preprocessing.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(pattern, symbols)
/usr/local/lib/python3.9/site-packages/texthero/preprocessing.py:131: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(rf"([{string.punctuation}])+", symbol)


The signature of Reuters can be taken out.

In [28]:
t_df['clean_text'] = [x.rsplit("reuters", 1)[-1] for x in t_df["text"]]

In [29]:
t_df = t_df[['clean_text','target']]
#data.rename(columns={'gdp':'log(gdp)'}, inplace=True)
t_df.rename(columns = {'clean_text':'text'},inplace = True)

That marks the extraction of "true news"dataset.
Now, off to turn text to words, vectors and models.

Both the datsets be merged into one but randomly.

In [30]:
t_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11239 entries, 0 to 11238
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    11239 non-null  object
 1   target  11239 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 175.7+ KB


In [31]:
f_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7585 entries, 0 to 7584
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7585 non-null   object
 1   target  7585 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 118.6+ KB


In [32]:
t_df.head()

,text,target
0,head conservative republican faction u congre...,1
1,transgender people allowed first time enlist ...,1
2,special counsel investigation links russia pr...,1
3,comment mueller' office declined comment trum...,1
4,president donald trump called u postal servic...,1


In [33]:
f_df['text'] = hero.clean(f_df['text'])

/usr/local/lib/python3.9/site-packages/texthero/preprocessing.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(pattern, symbols)
/usr/local/lib/python3.9/site-packages/texthero/preprocessing.py:131: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(rf"([{string.punctuation}])+", symbol)


In [34]:
f_df.head()

,text,target
0,according hill democrat senator bob casey warn...,0
1,protect every government building public venue...,0
2,ami horowitz fantastic check man street effort...,0
3,one reminder president trump chose nonsense ge...,0
4,thank goodnesss jeff sessions moving finding w...,0


In [35]:
tf_df = pd.concat([t_df,f_df], axis = 0, ignore_index = True)

In [36]:
tf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18824 entries, 0 to 18823
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    18824 non-null  object
 1   target  18824 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 294.2+ KB


In [37]:
tf_df.head()

,text,target
0,head conservative republican faction u congre...,1
1,transgender people allowed first time enlist ...,1
2,special counsel investigation links russia pr...,1
3,comment mueller' office declined comment trum...,1
4,president donald trump called u postal servic...,1


In [38]:
tf_df = tf_df.sample(frac = 1)

In [39]:
tf_df.reset_index(drop = True)

,text,target
0,u advisory commission warned wednesday china'...,1
1,summer spat president donald trump senate maj...,1
2,live feed turkey coup,0
3,anyone remember girls dixie days release drug ...,0
4,head democratic party resigned sunday amid fu...,1
...,...,...
18819,immediately president trump speech horrible ma...,0
18820,ipsos polling events planned slew democrat li...,1
18821,21st century wire says imagine nations like ch...,0
18822,editorial staff sap editorial involvement cre...,1


Well, sampling gave 5 extra rows. Is there a better way to shufflel the rows
Also, there is s preceding 'u'for some of the true news datsets.


train/test split

In [40]:
X = tf_df[['text']]
y = tf_df[['target']]


In [41]:
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.33, random_state=42)

In [42]:
X_train.head()

,text
5405,u senate judiciary committee hold confirmatio...
3378,"""raise lot money knock lot doors win lot seat..."
12402,soooo irs lied americans prod comply sign obam...
3735,bettors sides atlantic ramping wagers niche o...
8715,close bitter congressional race mike pence ca...


In [43]:
y_train.head()

,target
5405,1
3378,1
12402,0
3735,1
8715,1


CountVectorizer

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train.text)
X_train_counts.shape

(12612, 57756)

In [45]:
count_vect.vocabulary_.get(u'algorithm')

3083

TFIDF Transformer

In [46]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(12612, 57756)

In [47]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(12612, 57756)

Model 1 - Maive Bayes Classifier

In [48]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train.target)

In [49]:
X_test_counts = count_vect.transform(X_test.text)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

predicted = clf.predict(X_test_tfidf)

In [50]:
np.mean(predicted == y_test.target)


0.8539922730199614

In [51]:
from sklearn import metrics
print(metrics.classification_report(y_test.target, predicted))

              precision    recall  f1-score   support

           0       0.98      0.65      0.78      2516
           1       0.81      0.99      0.89      3696

    accuracy                           0.85      6212
   macro avg       0.89      0.82      0.84      6212
weighted avg       0.88      0.85      0.85      6212



In [52]:
metrics.confusion_matrix(y_test.target, predicted)

array([[1644,  872],
       [  35, 3661]])

It is not ablle to clearly identify some fake news items yet, just like humans. 

Model 2 - Logistic Regression Classifier

In [53]:
#Logistic regression
from sklearn.linear_model import LogisticRegression
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
model=scikit_log_reg.fit(X_train_tfidf,y_train)

/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[LibLinear]

In [54]:
from sklearn import metrics
probs = model.predict_proba(X_test_tfidf)
preds = model.predict(X_test_tfidf)
score = model.score(X_test_tfidf, y_test)
cm = metrics.confusion_matrix(y_test, preds)
cr=metrics.classification_report(y_test, model.predict(X_test_tfidf))
print(score)
print(cm)
print(cr)

0.9634578235672892
[[2403  113]
 [ 114 3582]]
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      2516
           1       0.97      0.97      0.97      3696

    accuracy                           0.96      6212
   macro avg       0.96      0.96      0.96      6212
weighted avg       0.96      0.96      0.96      6212



That seems like an excellent model! The dataset has very distinct features that separate the classes of True and Fake within the political news category and only news stories ( without twitter data). It would be a good idea to see if the entire dataset amongst all news categories would perform as well. The motivation to build a better model would be drawn from that behavior.
But let u scontinue to explore more models like random forest, shallow neuraal and deep neural.


Model3 - Logistic regression with optimization - SGDClassifier

In [55]:
from sklearn.linear_model import SGDClassifier
clf= SGDClassifier( class_weight='balanced', alpha=0.3, penalty='l2', loss='log', random_state=42)
clf.fit(X_train_tfidf, y_train.target)
predicted_sgdc = clf.predict(X_test_tfidf)
np.mean(predicted_sgdc == y_test.target)

0.6001287830006439

In [56]:
score = clf.score(X_test_tfidf, y_test)
cm = metrics.confusion_matrix(y_test.target, predicted_sgdc)
cr=metrics.classification_report(y_test.target, model.predict(X_test_tfidf))
print(score)
print(cm)
print(cr)

0.6001287830006439
[[  32 2484]
 [   0 3696]]
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      2516
           1       0.97      0.97      0.97      3696

    accuracy                           0.96      6212
   macro avg       0.96      0.96      0.96      6212
weighted avg       0.96      0.96      0.96      6212



MOdel 4 - SVM Classifier

In [57]:
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train.target)

SVC(gamma='auto', kernel='linear')

In [58]:
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test_tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test.target)*100)

SVM Accuracy Score ->  96.08821635544108


In [59]:
score_SVM = SVM.score(X_test_tfidf, y_test)
cm_SVM = metrics.confusion_matrix(y_test.target, predictions_SVM)
cr_SVM=metrics.classification_report(y_test.target, SVM.predict(X_test_tfidf))
print(score_SVM)
print(cm_SVM)
print(cr_SVM)

0.9608821635544108
[[2401  115]
 [ 128 3568]]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      2516
           1       0.97      0.97      0.97      3696

    accuracy                           0.96      6212
   macro avg       0.96      0.96      0.96      6212
weighted avg       0.96      0.96      0.96      6212



Logistic regression is very slightly better but certainly faster.

In [ ]:
#Future anticipattion

Model 6 - RNN

In [ ]:
#To be continued in future.

Resources:
1. https://kavita-ganesan.com/news-classifier-with-logistic-regression-in-python/#.YLnfZ2bYq_t
2.https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/
